# MNS - Biological Plausible Deep Learning
## Run Guergiev et al (2017) Pipeline

In [58]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


* **General Structure of Network Module**

    * Init: Declare layer shapes, data, define spike history buffer, init weights/layers
    * init_weights: Loops over all layers and initializes weights
       * Hyperparameters
            - use_weight_optimization: Desired avg/sd of somatic potentials
            - use_feedback_bias: Not in paper - not only synapses Y but also intercept
            - use_sparse_feedback: 80% of weights randomly set to 0
            - use_broadcast: feedback to all layers comes from output layer
        * TODOs
            - Don't loop over layers but sample ones a large set
            - Add dropout percentage and magnitude increase vars
            
    * make_weights_symmetric: 
        - Hyperparameters: Feedback weights = Transposes feedforward weights
            - noisy_symmetric_weights: Add N(0, 0.05) noise to data
            - use_broadcast: feedback to all layers comes from output layer
        - TODOs
            - use_broadcast only needed with more than one hidden layer!
    * init_layers: Loops over layers and adds them to l object in class
    * out_f: Forward phase pass through net - calls out_f of different layers in loop
    * out_t: Target phase pass through net - adds target at top layer
        - TODOs
            - merge out_f and out_t - None vs self.t
    * f_phase: Forward phase
    * t_phase: Target phase
    * train: Loop over epochs and loop over all examples
    * test_weights

In [59]:
from sklearn.model_selection import train_test_split

# Import log-helper/learning plot functions
from utils.helpers import *
from utils.plotting import *

# Guergiev et al (2017) - Segregated Compartments DNN Learning
from models.CompDNN import CompDNN, eval_comp_dnn

In [60]:
# Create all necessary directory if non-existent
global data_dir
data_dir = os.getcwd() +"/data"

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print("Created New Data Directory")
    
download_data()

No download of MNIST needed.
No download of Fashion-MNIST needed.
No download of CIFAR-10 needed.


In [74]:
# MNIST dataset
X_mnist, y_mnist = get_data(num_samples=70000, dataset="mnist")

X_train, X_test, y_train, y_test = train_test_split(X_mnist, y_mnist,
                                                    stratify=y_mnist,
                                                    random_state=0,
                                                    test_size=1./7)

# set training parameters
f_etas = (0.21, 0.21)
b_etas = None
n_epochs = 10

n=(500, 10)

net = CompDNN(n, X_train, y_train)

net.train(f_etas, b_etas, n_epochs,
          log_freq=5000, verbose=True, logging=True, dataset="mnist")

(60000, 1, 28, 28) (60000,)


In [ ]:
X_test, y_test = prep_data_guergiev(X_test, y_test)
acc, loss = net.get_test_error(X_test, y_test)
print("Test Accuracy MNIST: {}".format(acc))

In [ ]:
# Fashion-MNIST dataset
X_fashion, y_fashion = get_data(num_samples=70000, dataset="fashion")

X_train, X_test, y_train, y_test = train_test_split(X_fashion, y_fashion,
                                                    stratify=y_fashion,
                                                    random_state=0,
                                                    test_size=1./7)

# set training parameters
f_etas = (0.21, 0.21)
b_etas = None
n_epochs = 10

n=(500, 10)

net = CompDNN(n, X_train, y_train)

net.train(f_etas, b_etas, n_epochs,
          log_freq=5000, verbose=True, logging=True, dataset="fashion")

In [ ]:
X_test, y_test = prep_data_guergiev(X_test, y_test)
acc, loss = net.get_test_error(X_test, y_test)
print("Test Accuracy Fashion-MNIST: {}".format(acc))

In [ ]:
# CIFAR-10 dataset
X_cifar10, y_cifar10 = get_data(num_samples=70000, dataset="cifar10")

X_train, X_test, y_train, y_test = train_test_split(X_cifar10, y_cifar10,
                                                    stratify=y_cifar10,
                                                    random_state=0,
                                                    test_size=1./7)

# set training parameters
f_etas = (0.21, 0.21)
b_etas = None
n_epochs = 10

n=(500, 10)
net = CompDNN(n, X_train, y_train)

net.train(f_etas, b_etas, n_epochs,
          log_freq=5000, verbose=True, logging=True, dataset="cifar10")

In [ ]:
X_test, y_test = prep_data_guergiev(X_test, y_test)
acc, loss = net.get_test_error(X_test, y_test)
print("Test Accuracy CIFAR-10: {}".format(acc))

## Bayesian Optimization

In [79]:
eval_comp_dnn("mnist", f_etas, b_etas, num_layers=1,
              h_l_1=500, h_l_2=0, h_l_3=0, h_l_4=0, h_l_5=0, h_l_6=0,
              num_epochs=2, k_fold=3, verbose=True)

Dataset: mnist
Learning Rate Weights: (0.21, 0.21)
Architecture of Cross-Validated Network:
	 Layer 0: 500 Units
	 Layer 1: 10 Units
Cross-Validation Score Fold 1: 0.9425351388412753
Cross-Validation Score Fold 2: 0.9206702952899327
Cross-Validation Score Fold 3: 0.9402083065449403


0.9344712468920494

In [ ]:
# Run Bayesian Optimization (UCB-Acquisition Fct) on DNN
hyper_space_comp_dnn = {'batch_size': (10, 500),
                        'learning_rate': (0.01, 0.5),
                        'num_layers': (1, 6),
                        'h_l_1': (30, 500),
                        'h_l_2': (30, 500),
                        'h_l_3': (30, 500),
                        'h_l_4': (30, 500),
                        'h_l_5': (30, 500),
                        'h_l_6': (30, 500)}

In [ ]:
use_sparse_feedback = False # silence 80% of feedback weights
nonspiking_mode = True  # Non-spiking mode (real-valued outputs)
use_conductances = True  # use conductances between dendrites and soma
use_broadcast = True  # use broadcast - feedback to all layers from output l
use_spiking_feedback = True  # use spiking feedback
use_spiking_feedforward = True  # use spiking feedforward input
use_symmetric_weights = False  # enforce symmetric weights
noisy_symmetric_weights = False  # add noise to symmetric weights

"""
Neurophysiological Hyperparameters
"""
lambda_max = 0.2*dt  # maximum spike rate (spikes per time step)

# kernel parameters
tau_s = 3.0  # synaptic time constant
tau_L = 10.0  # leak time constant

# conductance parameters
g_B = 0.6  # basal conductance
g_A = 0.05 if use_apical_conductance else 0  # apical conductance
g_L = 1.0/tau_L  # leak conductance
g_D = g_B  # dendritic conductance in output layer

# reversal potential parameters
E_E = 8  # excitation
E_I = -8  # inhibition

# steady state constants
k_B = g_B/(g_L + g_B + g_A)
k_D = g_D/(g_L + g_D)
k_I = 1.0/(g_L + g_D)

# weight update constants
P_hidden = 20.0/lambda_max      # hidden layer error signal scaling factor
P_final = 20.0/(lambda_max**2)  # final layer error signal scaling factor